In [1]:
import pandas as pd
import numpy as np
import json

cols_needed = ['attacktype1_txt', 'success', 'nkill', 'nwound', 'property', 'propvalue']
df = pd.read_csv('terrorism_dataset.csv', encoding='ISO-8859-1', usecols=cols_needed, low_memory=False)

target_attacks = [
    'Bombing/Explosion',
    'Armed Assault',
    'Assassination',
    'Hostage Taking (Kidnapping)',
    'Facility/Infrastructure Attack'
]
df = df[df['attacktype1_txt'].isin(target_attacks)].copy()

cols_standard = ['success', 'nkill', 'nwound', 'attacktype1_txt']
df_clean = df.dropna(subset=cols_standard).copy()

df_clean['propvalue'] = pd.to_numeric(df_clean['propvalue'], errors='coerce')
df_clean.loc[df_clean['property'] != 1, 'propvalue'] = np.nan

features_chart = ['success', 'nkill', 'nwound', 'propvalue']
grouped = df_clean.groupby('attacktype1_txt')[features_chart].mean()
global_max_values = grouped.max()
avg_attack_global = df_clean[features_chart].mean()

output_data = {
    "meta": {
        "description": "Dati per Radar Chart D3.js",
        "features": [
            {"key": "success", "label": "Success Rate"},
            {"key": "nkill", "label": "Avg Kills"},
            {"key": "nwound", "label": "Avg Wounded"},
            {"key": "propvalue", "label": "Avg Damage ($)"}
        ],
        "global_max_values": {
            "success_rate": float(global_max_values['success']),
            "avg_kills": float(global_max_values['nkill']),
            "avg_wounded": float(global_max_values['nwound']),
            "avg_damage": float(global_max_values['propvalue'])
        },
        "global_average": {
            "success_rate": float(avg_attack_global['success']),
            "avg_kills": float(avg_attack_global['nkill']),
            "avg_wounded": float(avg_attack_global['nwound']),
            "avg_damage": float(avg_attack_global['propvalue'])
        }
    },
    "data": []
}

for attack_type in target_attacks:
    if attack_type in grouped.index:
        row = grouped.loc[attack_type]
        entry = {
            "attack_type": attack_type,
            "metrics": {
                "success_rate": float(row['success']),
                "avg_kills": float(row['nkill']),
                "avg_wounded": float(row['nwound']),
                "avg_damage": float(row['propvalue']) if not pd.isna(row['propvalue']) else 0.0
            }
        }
        output_data["data"].append(entry)

with open('radar_chart.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2)